In [8]:
import os
import json

from PIL import Image
from scipy import ndimage
import math
import json
import torch
import numpy as np
import pandas as pd

import cv2
import matplotlib

from matplotlib import pyplot as plt

from tqdm import tqdm
from pathlib import Path
from dotenv import load_dotenv

from utils.yolo import get_teeth_ROI
from utils.edge import tooth_isolation
from utils.preprocess import get_labels_by_image
from utils.preprocess import xyxy_reformat, rotate_bounding_boxes, rect_include_another

matplotlib.use('module://matplotlib_inline.backend_inline')
load_dotenv()

True

In [ ]:
model = torch.hub.load(r'..\YOLO', 'custom', path=r'..\YOLO\weights\8-bound.pt', source='local')
data_dir = '..' / Path(os.getenv('DATASET_DIR'))


YOLOv5  torch 1.12.0 CUDA:0 (NVIDIA GeForce RTX 2070, 8192MiB)



requirements: D:\Users\douli\anaconda3\envs\dentist-CV-main\Lib\site-packages\yolov5\requirements.txt not found, check failed.


In [11]:
image_names = list(data_dir.glob('*.jpg'))
crop_dir = '.' / Path('region_crop')
meta_data = {}

for image_name in tqdm(image_names):
    results = model(image_name)
    teeth_roi = get_teeth_ROI(results)
    teeth_roi_images = teeth_roi['images'][image_name.stem]

    count = 0
    for target_roi in teeth_roi_images:
        flag = target_roi['flag']
        tooth_position = target_roi['tooth_position']
        target_roi_image = target_roi['image']

        filename = f'{image_name.stem} {count}.jpg'
        # save_filepath = crop_dir / filename
        count += 1

        meta_data[filename] = target_roi

        # temp_im = Image.fromarray(target_roi_image)
        # temp_im.save(save_filepath)

with open('region_data.json', 'w') as f:
    json.dump(meta_data, f)


100%|██████████| 600/600 [00:48<00:00, 12.31it/s]


TypeError: Object of type ndarray is not JSON serializable

In [13]:
meta_data['00008228 1.jpg']


{'flag': 'upper',
 'number': 1,
 'tooth_position': 'middle',
 'org_file_name': '00008228',
 'offset': array([1017,  295]),
 'image': array([[[ 98,  98,  98],
         [ 98,  98,  98],
         [ 97,  97,  97],
         ...,
         [164, 164, 164],
         [166, 166, 166],
         [168, 168, 168]],
 
        [[101, 101, 101],
         [101, 101, 101],
         [101, 101, 101],
         ...,
         [178, 178, 178],
         [179, 179, 179],
         [179, 179, 179]],
 
        [[111, 111, 111],
         [113, 113, 113],
         [112, 112, 112],
         ...,
         [191, 191, 191],
         [189, 189, 189],
         [187, 187, 187]],
 
        ...,
 
        [[131, 131, 131],
         [131, 131, 131],
         [129, 129, 129],
         ...,
         [ 48,  48,  48],
         [ 45,  45,  45],
         [ 44,  44,  44]],
 
        [[126, 126, 126],
         [125, 125, 125],
         [120, 120, 120],
         ...,
         [ 53,  53,  53],
         [ 51,  51,  51],
         [ 50,  5